## **Import**

In [1]:
from sqlalchemy import create_engine, MetaData, text, inspect
from sqlalchemy.schema import CreateTable
import pandas as pd
from dotenv import load_dotenv
import os
import re
import subprocess

import subprocess
import psycopg2
import os

## **Input Parameters**

In [2]:
# Cargar variables desde .env
load_dotenv()

# Obtener los valores
SOURCE_USER = os.getenv("SOURCE_DB_USER")
SOURCE_PASSWORD = os.getenv("SOURCE_DB_PASSWORD")
SOURCE_HOST = os.getenv("SOURCE_DB_HOST")
SOURCE_PORT = os.getenv("SOURCE_DB_PORT")
SOURCE_DATABASE = os.getenv("SOURCE_DB_NAME")

TARGET_USER = os.getenv("TARGET_DB_USER")
TARGET_PASSWORD = os.getenv("TARGET_DB_PASSWORD")
TARGET_HOST = os.getenv("TARGET_DB_HOST")
TARGET_PORT = os.getenv("TARGET_DB_PORT")
TARGET_DATABASE = os.getenv("TARGET_DB_NAME")

## **Functions**

In [3]:
def dql_execution(query_text, engine):
    df = pd.read_sql(query_text, engine)
    print(df)

In [4]:
def ddl_dml_execution(sql_string,engine):
    with engine.connect() as connection:
        connection.execute(text(sql_string))
        connection.commit()  

In [10]:
def get_database_ddl(source_schemas,source_engine,source_inspector,target_engine):
    
    ddl_strings = {}

    for schema_name in source_schemas:
        metadata = MetaData()
        metadata.reflect(bind=source_engine,schema=schema_name)
        print(metadata.schema)

## **Process**

In [11]:
source_engine = create_engine(f"postgresql://{SOURCE_USER}:{SOURCE_PASSWORD}@{SOURCE_HOST}:5432/{SOURCE_DATABASE}")
target_engine = create_engine(f"postgresql://{TARGET_USER}:{TARGET_PASSWORD}@{TARGET_HOST}:5433/{TARGET_DATABASE}")

In [12]:
source_inspector = inspect(source_engine)

In [16]:
source_schemas = source_inspector.get_schema_names()
source_schemas

['public', 'university']

In [14]:
source_schemas.remove("information_schema")

In [15]:
ddl_dict = get_database_ddl(source_schemas=source_schemas,source_engine=source_engine,source_inspector=source_inspector,target_engine=target_engine)

None
None


In [93]:
print(ddl_dict)

{'course': {'create': '\nCREATE TABLE IF NOT EXISTS university.course (\n\tcourseno CHAR(6) NOT NULL, \n\tcrsdesc VARCHAR(50) NOT NULL, \n\tcrsunits INTEGER)\n\n;', 'constraints': 'ALTER TABLE university.course  \n\tADD CONSTRAINT coursepk PRIMARY KEY (courseno), \n\tADD CONSTRAINT uniquecrsdesc UNIQUE (crsdesc)\n;'}, 'faculty': {'create': '\nCREATE TABLE IF NOT EXISTS university.faculty (\n\tfacno CHAR(11) NOT NULL, \n\tfacfirstname VARCHAR(30) NOT NULL, \n\tfaclastname VARCHAR(30) NOT NULL, \n\tfaccity VARCHAR(30) NOT NULL, \n\tfacstate CHAR(2) NOT NULL, \n\tfaczipcode CHAR(10) NOT NULL, \n\tfacrank CHAR(4), \n\tfachiredate DATE, \n\tfacsalary NUMERIC(10, 2), \n\tfacsupervisor CHAR(11), \n\tfacdept CHAR(6))\n\n;', 'constraints': 'ALTER TABLE university.faculty  \n\tADD CONSTRAINT facultypk PRIMARY KEY (facno), \n\tADD CONSTRAINT supervisorfk FOREIGN KEY(facsupervisor) REFERENCES university.faculty (facno)\n;'}, 'student': {'create': '\nCREATE TABLE IF NOT EXISTS university.student (\

In [94]:
for key,value in ddl_dict.items():
    ddl_dml_execution(value['create'],target_engine)

In [95]:
for key,value in ddl_dict.items():
    ddl_dml_execution(value['constraints'],target_engine)